Competion Name:
# **BigQuery AI - Building the Future of Data**
Submission Title:
# **VectorVault: Intelligent Data/Knowledge Base System**
*Submitted by - Yash Tamgadge*

# About the code/notebook and instructions
**Code in a nutshell -**
1. Reads different types of files
2. Converts to Vector DB
3. Enhanced knowledge
   1. **Clusters vectors, forms Knowledge Graphs**
   2. *Each doc could be part of multiple clusters*
4. **Point 3 becomes crucial to find hidden / related knowledge**
5. User queries
6. Gets output

**Instructions**
1. The code automatically reads all files
2. Please run the cell below and select 1 to query and 2 to see db stats
3. Please enter one of the sample queries to see output

**About the data set**
*

In [1]:
#please run the cell below to install libraries
#feel free to select show code to look at the code

In [2]:
# @title
#install libraries
!pip install pypdf2
!pip install pytesseract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [3]:
#please run the cell below to run the actual code
#you will get an interactive output
#feel free to select show code to look at the code

In [ ]:
# @title
#!/usr/bin/env python3
"""
Comprehensive Document Processing System with Vector Embeddings and Knowledge Graph
Supports: PDF, TXT, XLSX, CSV, Images, JSON
Features: Chunk-level Clustering, Vector DB, NER, Knowledge Graph
"""

# Import the warnings library and disable all warnings
import warnings
warnings.filterwarnings('ignore')

import os
import json
import sqlite3
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Tuple
from collections import defaultdict
import pickle
import requests
import re

# Core libraries import
import PyPDF2
import openpyxl
from PIL import Image
import pytesseract
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from transformers import pipeline
from sentence_transformers import SentenceTransformer

print("✓ All libraries loaded successfully")

def download_file_from_drive(file_id, destination):
    """Download file from Google Drive using file ID"""
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()

    response = session.get(URL, params={'id': file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': file_id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

class DocumentProcessor:
    """Main class for document processing and vector database management"""

    def __init__(self, db_path: str = "/content/document_system.db"):
        self.db_path = db_path
        self.embedding_generator = None
        self.ner_pipeline = None
        self.embeddings = [] # This will now store chunk embeddings and metadata
        self.clusters = {}
        self.cluster_centroids = {}
        self.knowledge_graph = nx.Graph()
        self.entity_embeddings = {}

        self._initialize_database()
        self._initialize_models()

    def _initialize_database(self):
        """Initialize SQLite database for document and chunk tracking"""
        conn = sqlite3.connect(self.db_path)
        db_cursor = conn.cursor()

        # Documents table (no longer stores cluster info)
        db_cursor.execute('''
            CREATE TABLE IF NOT EXISTS documents (
                pk INTEGER PRIMARY KEY,
                filename TEXT NOT NULL,
                file_type TEXT NOT NULL,
                content TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')

        # Embeddings table (stores chunks, embeddings, and cluster IDs)
        db_cursor.execute('''
            CREATE TABLE IF NOT EXISTS embeddings (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                document_pk INTEGER,
                embedding BLOB,
                cluster_id TEXT,
                chunk_id INTEGER DEFAULT 0,
                chunk_content TEXT,
                FOREIGN KEY (document_pk) REFERENCES documents (pk)
            )
        ''')

        # Entities table
        db_cursor.execute('''
            CREATE TABLE IF NOT EXISTS entities (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                entity TEXT NOT NULL,
                entity_type TEXT,
                document_pk INTEGER,
                embedding BLOB,
                FOREIGN KEY (document_pk) REFERENCES documents (pk)
            )
        ''')

        conn.commit()
        conn.close()

    def _initialize_models(self):
        """Initialize embedding generator and NER pipeline"""
        print("Initializing models...")
        self.embedding_generator = SentenceTransformer('all-MiniLM-L6-v2')
        print("✓ Embedding generator initialized.")

        self.ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")
        print("✓ Transformers NER pipeline loaded.")

    def read_pdf(self, file_path: str) -> str:
        text = ""
        with open(file_path, 'rb') as file:
            try:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
            except Exception as e:
                print(f"Warning: Could not read PDF {file_path}. Error: {e}")
        return text.strip()

    def read_xlsx(self, file_path: str) -> str:
        try:
            df = pd.read_excel(file_path, engine='openpyxl')
            return df.to_string()
        except Exception as e:
            print(f"Warning: Could not read XLSX {file_path}. Error: {e}")
            return ""

    def read_csv(self, file_path: str) -> str:
        try:
            df = pd.read_csv(file_path)
            return df.to_string()
        except Exception as e:
            print(f"Warning: Could not read CSV {file_path}. Error: {e}")
            return ""

    def read_txt(self, file_path: str) -> str:
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                return file.read()
        except Exception as e:
            print(f"Warning: Could not read TXT {file_path}. Error: {e}")
            return ""

    def read_json(self, file_path: str) -> str:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
            return json.dumps(data, indent=2)
        except Exception as e:
            print(f"Warning: Could not read JSON {file_path}. Error: {e}")
            return ""

    def read_image(self, file_path: str) -> str:
        try:
            # You might need to specify the path to tesseract executable
            # pytesseract.pytesseract.tesseract_cmd = r'<full_path_to_your_tesseract_executable>'
            image = Image.open(file_path)
            text = pytesseract.image_to_string(image)
            return text.strip()
        except Exception as e:
            print(f"Warning: OCR failed for {file_path}. Is Tesseract installed? Error: {e}")
            return ""

    def read_document(self, file_path: str) -> str:
        file_path = Path(file_path)
        extension = file_path.suffix.lower()
        readers = {
            '.pdf': self.read_pdf, '.txt': self.read_txt, '.xlsx': self.read_xlsx,
            '.csv': self.read_csv, '.json': self.read_json, '.jpg': self.read_image,
            '.jpeg': self.read_image, '.png': self.read_image
        }
        reader = readers.get(extension)
        if reader:
            return reader(str(file_path))
        print(f"Warning: Unsupported file type: {extension}")
        return ""

    def generate_embedding(self, text: str) -> np.ndarray:
        if not text or not text.strip():
            return np.zeros(self.embedding_generator.get_sentence_embedding_dimension())
        return self.embedding_generator.encode(text)

    def extract_entities(self, text: str) -> List[Tuple[str, str]]:
        if not text.strip() or len(text) < 10:
            return []

        # The model has a max sequence length, so we process in chunks if necessary.
        max_len = 512
        text_chunks = [text[i:i+max_len] for i in range(0, len(text), max_len)]

        entities = []
        for chunk in text_chunks:
            try:
                ner_results = self.ner_pipeline(chunk)
                for entity in ner_results:
                    if entity['score'] > 0.8: # Higher confidence threshold
                        word = entity['word'].replace('##', '').strip()
                        if len(word) > 2: # Filter out short/irrelevant entities
                            entities.append((word, entity['entity_group']))
            except Exception as e:
                print(f"NER processing failed for a chunk: {e}")

        # Return unique entities
        return list(set(entities))

    def chunk_text(self, text: str, chunk_size: int = 250, overlap: int = 25) -> List[str]:
        if not text or len(text.strip()) < 10:
            return []

        text = re.sub(r'\s+', ' ', text.strip())
        chunks = []
        for i in range(0, len(text), chunk_size - overlap):
            chunk = text[i:i + chunk_size]
            if chunk.strip():
                chunks.append(chunk.strip())
        return [chunk for chunk in chunks if len(chunk.strip()) > 20]

    def process_documents(self, file_paths: List[str]):
        print(f"\n🔄 Processing {len(file_paths)} documents...")
        self.embeddings = []
        all_doc_entities = []

        conn = sqlite3.connect(self.db_path)
        db_cursor = conn.cursor()

        for i, file_path in enumerate(file_paths, 1):
            filename = os.path.basename(file_path)
            print(f"📄 [{i}/{len(file_paths)}] Processing: {filename}")

            content = self.read_document(file_path)
            if not content or len(content.strip()) < 10:
                print(f"   -> Skipped: No content extracted from {filename}")
                continue

            file_type = Path(file_path).suffix.lower()
            db_cursor.execute('INSERT INTO documents (filename, file_type, content) VALUES (?, ?, ?)',
                              (filename, file_type, content))
            doc_pk = db_cursor.lastrowid

            chunks = self.chunk_text(content)
            print(f"   -> Created {len(chunks)} chunks.")

            for chunk_id, chunk in enumerate(chunks):
                chunk_embedding = self.generate_embedding(chunk)
                self.embeddings.append({
                    'embedding': chunk_embedding,
                    'document_pk': doc_pk,
                    'chunk_id': chunk_id,
                    'content': chunk,
                    'filename': filename
                })
                embedding_blob = pickle.dumps(chunk_embedding)
                db_cursor.execute('''INSERT INTO embeddings
                                     (document_pk, embedding, chunk_id, chunk_content)
                                     VALUES (?, ?, ?, ?)''',
                                  (doc_pk, embedding_blob, chunk_id, chunk))

            entities = self.extract_entities(content)
            all_doc_entities.append(entities)
            for entity, entity_type in entities:
                entity_embedding = self.generate_embedding(entity)
                db_cursor.execute('INSERT INTO entities (entity, entity_type, document_pk, embedding) VALUES (?, ?, ?, ?)',
                                  (entity, entity_type, doc_pk, pickle.dumps(entity_embedding)))
            print(f"✓ Stored document and {len(chunks)} chunks (PK: {doc_pk}). Extracted {len(entities)} entities.")

        conn.commit()
        conn.close()

        if self.embeddings:
            print("\n🔬 Clustering chunks...")
            self.cluster_chunks(n_clusters=min(10, len(self.embeddings) // 5 + 1))
            print("\n🕸️ Building knowledge graph...")
            self.build_knowledge_graph(all_doc_entities)
        else:
            print("No documents were processed successfully.")

    def cluster_chunks(self, n_clusters: int):
        if len(self.embeddings) < n_clusters or n_clusters < 2:
            print("Not enough chunks to form multiple clusters.")
            return

        embedding_vectors = np.array([e['embedding'] for e in self.embeddings])
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(embedding_vectors)

        self.cluster_centroids = {f"c{i}": centroid for i, centroid in enumerate(kmeans.cluster_centers_)}

        conn = sqlite3.connect(self.db_path)
        db_cursor = conn.cursor()

        for i, label in enumerate(cluster_labels):
            cluster_id = f"c{label}"
            # Update in-memory list
            self.embeddings[i]['cluster_id'] = cluster_id
            # Update database
            doc_pk = self.embeddings[i]['document_pk']
            chunk_id = self.embeddings[i]['chunk_id']
            db_cursor.execute('UPDATE embeddings SET cluster_id = ? WHERE document_pk = ? AND chunk_id = ?',
                              (cluster_id, doc_pk, chunk_id))

        conn.commit()
        conn.close()
        print(f"✓ Clustered {len(self.embeddings)} chunks into {n_clusters} clusters.")

    def build_knowledge_graph(self, all_doc_entities: List[List[Tuple[str, str]]]):
        for doc_entities in all_doc_entities:
            for entity, entity_type in doc_entities:
                self.knowledge_graph.add_node(entity, type=entity_type)
                if entity not in self.entity_embeddings:
                    self.entity_embeddings[entity] = self.generate_embedding(entity)

            entities_in_doc = [entity for entity, _ in doc_entities]
            for i in range(len(entities_in_doc)):
                for j in range(i + 1, len(entities_in_doc)):
                    e1, e2 = entities_in_doc[i], entities_in_doc[j]
                    if self.knowledge_graph.has_edge(e1, e2):
                        self.knowledge_graph[e1][e2]['weight'] += 1
                    else:
                        self.knowledge_graph.add_edge(e1, e2, weight=1)
        print(f"✓ Knowledge graph created with {self.knowledge_graph.number_of_nodes()} nodes and {self.knowledge_graph.number_of_edges()} edges.")

    def find_most_relevant_chunks(self, query: str, top_k: int = 10) -> List[Dict[str, Any]]:
        if not self.embeddings:
            return []

        query_embedding = self.generate_embedding(query)

        chunk_similarities = []
        for emb_data in self.embeddings:
            similarity = cosine_similarity([query_embedding], [emb_data['embedding']])[0][0]
            chunk_similarities.append({**emb_data, 'similarity': similarity})

        chunk_similarities.sort(key=lambda x: x['similarity'], reverse=True)
        return chunk_similarities[:top_k]

    def generate_smart_answer(self, query: str, relevant_chunks: List[Dict[str, Any]]) -> Dict[str, Any]:
        if not relevant_chunks:
            return {"answer": "No relevant information found.", "sources": []}

        query_keywords = set(query.lower().split())

        scored_chunks = []
        for chunk in relevant_chunks:
            content_lower = chunk['content'].lower()
            keyword_matches = sum(1 for keyword in query_keywords if keyword in content_lower)
            keyword_density = keyword_matches / len(content_lower.split()) if content_lower.split() else 0

            # Refined scoring: similarity is key, density is a tie-breaker
            combined_score = (chunk['similarity'] * 0.85) + (keyword_density * 0.15)
            scored_chunks.append({**chunk, 'score': combined_score})

        scored_chunks.sort(key=lambda x: x['score'], reverse=True)

        answer_parts = []
        used_sources = set()
        used_chunks = set()
        answer_length = 0
        max_length = 500

        for chunk in scored_chunks:
            if answer_length > max_length:
                break

            # Avoid adding nearly identical chunks
            is_too_similar = False
            for used_content in used_chunks:
                if cosine_similarity(
                    [self.generate_embedding(chunk['content'])],
                    [self.generate_embedding(used_content)]
                )[0][0] > 0.9:
                    is_too_similar = True
                    break
            if is_too_similar:
                continue

            answer_parts.append(chunk['content'])
            used_sources.add(chunk['filename'])
            used_chunks.add(chunk['content'])
            answer_length += len(chunk['content'])

        full_answer = " ".join(answer_parts)
        full_answer = re.sub(r'\s+', ' ', full_answer).strip()

        return {"answer": full_answer, "sources": list(used_sources), "top_chunk": scored_chunks[0]}

    def query_system(self, query: str) -> Dict[str, Any]:
        print(f"\n🔍 Processing query: '{query}'")

        if not self.embeddings:
            return {"answer": "No documents have been processed yet."}

        relevant_chunks = self.find_most_relevant_chunks(query)
        if not relevant_chunks:
            return {"answer": "No relevant information found in the documents."}

        print(f"📍 Found {len(relevant_chunks)} potentially relevant chunks.")
        for i, chunk in enumerate(relevant_chunks[:3]):
            print(f"   {i+1}. From '{chunk['filename']}' (Similarity: {chunk['similarity']:.3f})")

        answer_data = self.generate_smart_answer(query, relevant_chunks)
        top_chunk = answer_data['top_chunk']
        doc_pk = top_chunk['document_pk']

        conn = sqlite3.connect(self.db_path)
        db_cursor = conn.cursor()
        db_cursor.execute('SELECT entity FROM entities WHERE document_pk = ?', (doc_pk,))
        entities = [row[0] for row in db_cursor.fetchall()]
        db_cursor.execute('SELECT file_type FROM documents WHERE pk = ?', (doc_pk,))
        file_type = db_cursor.fetchone()[0]
        conn.close()

        result = {
            "answer": answer_data["answer"],
            "all_sources": answer_data["sources"],
            "primary_source_file": top_chunk['filename'],
            "primary_source_similarity": round(top_chunk['similarity'], 3),
            "related_entities": entities[:10]
        }
        return result

def main():
    print("=" * 60)
    print("📚 Document Processing System with Vector Embeddings")
    print("=" * 60)

    processor = DocumentProcessor()

    db_file = processor.db_path
    if os.path.exists(db_file):
        print(f"🧹 Clearing existing database '{db_file}'...")
        os.remove(db_file)
        processor._initialize_database()


    # --- OPTION 2: Download files from Google Drive (Original setup) ---
    print("📥 Downloading sample files from Google Drive...")
    file_downloads = {
         'AI_Creator_Tools.pdf': '1XkVCx-h_EEg0ZolrztEksw_XaD5S3eO0',
         'Creator_Economy_Ecosystem': '1WQmGrIt--E5Sai3hcDmUVsdnEBYg5zem',
        'Social_Media_Monteization.txt': '12pmeWCSPNZKXkC6OZGEYKP1MAssIa2kf'
     }

    file_paths = []
    for filename, file_id in file_downloads.items():
         local_path = f'/content/{filename}'
         print(f"   Downloading {filename}...")
         download_file_from_drive(file_id, local_path)
         file_paths.append(local_path)
         print(f"   ✅ {filename} downloaded")

    print(f"✅ Found {len(file_paths)} files ready for processing!")

    # Check if files exist before processing
    existing_files = [p for p in file_paths if os.path.exists(p)]
    if not existing_files:
         print("\n⚠️  WARNING: None of the specified files were found. Please create them.")
         print("   You can create the files in the /content/ directory with the provided content.")
    else:
        processor.process_documents(existing_files)
        print("\n✅ Document processing complete!")

    while True:
        print("\n" + "=" * 40)
        print("Choose an option:")
        print("1. Query documents")
        print("2. View database statistics")
        print("3. Exit")
        print("=" * 40)

        choice = input("\nEnter your choice (1-3): ").strip()

        if choice == "1":
            if not processor.embeddings:
                print("Cannot query because no documents have been processed successfully.")
                continue

            while True:
                query = input("\nEnter your query (or type 'back' to return): ").strip()
                if query.lower() == 'back': break

                if query:
                    result = processor.query_system(query)
                    print("\n" + "="*50 + "\n🎯 SEARCH RESULTS\n" + "="*50)
                    print(f"🗣️ Answer: {result.get('answer', 'N/A')}\n")
                    print(f"📂 Sources: {', '.join(result.get('all_sources', ['N/A']))}")
                    print(f"⭐ Primary Source: {result.get('primary_source_file')} (Similarity: {result.get('primary_source_similarity', 0)})")
                    if result.get('related_entities'):
                        print(f"🕸️ Related Entities: {', '.join(result['related_entities'])}")

        elif choice == "2":
            print("\n📊 Database Statistics")
            if not os.path.exists(processor.db_path):
                print("Database file does not exist. No documents processed.")
                continue

            conn = sqlite3.connect(processor.db_path)
            db_cursor = conn.cursor()
            doc_count = db_cursor.execute("SELECT COUNT(*) FROM documents").fetchone()[0]
            emb_count = db_cursor.execute("SELECT COUNT(*) FROM embeddings").fetchone()[0]
            ent_count = db_cursor.execute("SELECT COUNT(*) FROM entities").fetchone()[0]
            clust_dist = db_cursor.execute("SELECT cluster_id, COUNT(*) FROM embeddings WHERE cluster_id IS NOT NULL GROUP BY cluster_id").fetchall()
            conn.close()

            print(f"📄 Total documents: {doc_count}")
            print(f"🔢 Total embeddings/chunks: {emb_count}")
            print(f"🏷️ Total entities: {ent_count}")
            if clust_dist:
                print("\n📊 Cluster distribution (chunks per cluster):")
                for cluster, count in clust_dist:
                    print(f"   {cluster}: {count} chunks")

        elif choice == "3":
            print("\n👋 Goodbye!")
            break

        else:
            print("❌ Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


✓ All libraries loaded successfully
📚 Document Processing System with Vector Embeddings
Initializing models...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


✓ Transformers NER pipeline loaded.
🧹 Clearing existing database '/content/document_system.db'...
📥 Downloading sample files from Google Drive...
   ✅ AI_Creator_Tools.pdf downloaded
   ✅ Creator_Economy_Ecosystem downloaded
   ✅ Social_Media_Monteization.txt downloaded
✅ Found 3 files ready for processing!

🔄 Processing 3 documents...
📄 [1/3] Processing: AI_Creator_Tools.pdf
   -> Created 13 chunks.
✓ Stored document and 13 chunks (PK: 1). Extracted 7 entities.
📄 [2/3] Processing: Creator_Economy_Ecosystem
   -> Skipped: No content extracted from Creator_Economy_Ecosystem
📄 [3/3] Processing: Social_Media_Monteization.txt
   -> Created 16 chunks.
✓ Stored document and 16 chunks (PK: 2). Extracted 12 entities.

🔬 Clustering chunks...
✓ Clustered 29 chunks into 6 clusters.

🕸️ Building knowledge graph...
✓ Knowledge graph created with 19 nodes and 87 edges.

✅ Document processing complete!

Choose an option:
1. Query documents
2. View database statistics
3. Exit

Enter your choice (1-3):